### Aikasarjaharjoitustyö datojen poiminta ja yhdistäminen.  
Tässä tiedostossa Käsitellään ilmatieteenlaitoksen data, niin, että sitä voidaan käyttää lineaarisen regression tai muiden algoritmien kanssa.

### Tuodaan datan käsittelyssä ja kuvantamisessa tarvittavat kirjastot. 

In [14]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Säädatan lataus, sarakkeiden poiminta ja datan tarkastelu sekä käsittely

Ladataan tiedot dataframeen. Käytetään materiaalina toisessa yhteydessä hyödynnettyä viinidataa. Muunnetaan ilmatieteenlaitoksen tarjoama aikamuoto formaattiin vvvv-kk-pp tt:mm. Dropataan turhat sarakkeet ja vaihdetaan sarakejärjestys.

In [15]:
df = pd.read_excel('Temp_data.xlsx')
df = df.rename(columns={'Keskituulen nopeus [m/s]':'wind','Lämpötilan keskiarvo [°C]':'C','Aika [Paikallinen aika]':'Aika','[W/m2]':'Wperm2'}) #Muunnetaan sarakkeiden nimet

df['DS'] = df['Vuosi'].astype(str) + '-' + df['Kuukausi'].astype(str).str.zfill(2) + '-' + df['Päivä'].astype(str).str.zfill(2) + ' ' + df['Aika'] #Tehdään aikamuunnos
df=df.drop(['Havaintoasema','Vuosi','Kuukausi','Päivä','Aika','Ylin lämpötila [°C]','Alin lämpötila [°C]'], axis=1) #Dropataan turhat sarakkeet

df_C = df.loc[:, ['DS', 'C','wind','hPa','moist%','Wperm2']] # Vaihdetaan sarakejärjestys
print(df_C.head())

                 DS     C wind     hPa moist% Wperm2
0  2019-05-23 00:00  16.4  2.5  1007.1     82   -4.8
1  2019-05-23 01:00  15.6  2.1  1007.1     83   -5.2
2  2019-05-23 02:00  14.8  3.4  1007.6     90   -4.8
3  2019-05-23 03:00  14.8  4.5  1007.8     96   -3.7
4  2019-05-23 04:00  14.8  4.9    1008     97   -2.9


Tarkastetaan missä muodossa sarakkeiden data on.

In [16]:
print(df_C.dtypes)


DS        object
C         object
wind      object
hPa       object
moist%    object
Wperm2    object
dtype: object


Tehdään datatyyppien muunnokset.

In [17]:
df_C['DS'] = pd.to_datetime(df_C['DS'])
df_C['DS'] = df_C['DS'].dt.strftime('%Y-%m-%d %H:%M')
df_C.to_excel('C.xlsx', index=False)
print(df_C)

                     DS     C wind     hPa moist% Wperm2
0      2019-05-23 00:00  16.4  2.5  1007.1     82   -4.8
1      2019-05-23 01:00  15.6  2.1  1007.1     83   -5.2
2      2019-05-23 02:00  14.8  3.4  1007.6     90   -4.8
3      2019-05-23 03:00  14.8  4.5  1007.8     96   -3.7
4      2019-05-23 04:00  14.8  4.9    1008     97   -2.9
...                 ...   ...  ...     ...    ...    ...
41099  2024-01-31 19:00   1.4  4.6  1007.5    100   -0.4
41100  2024-01-31 20:00   1.7  6.6  1006.3    100   -0.3
41101  2024-01-31 21:00   1.6  8.9  1005.1     98   -0.8
41102  2024-01-31 22:00   1.2  7.7    1004     90   -0.3
41103  2024-01-31 23:00     1  7.2  1002.9     91   -0.4

[41104 rows x 6 columns]


Edellä olevat voidaan myös todentaa tarkastelemalla muutujien tilastollisia tietoja. Jokaisella muuttujalla on variaatiota. (std <> 0 )

In [18]:
df_C.describe()

,DS,C,wind,hPa,moist%,Wperm2
count,41104,41104.0,41104.0,41089.0,41089,41089.0
unique,41099,573.0,141.0,779.0,85,6506.0
top,2020-10-25 03:00,0.3,3.5,1016.5,100,-0.5
freq,2,279.0,841.0,202.0,3060,1090.0


Lämpötiladata näyttää olevan on OK. 